In [ ]:
%%capture

!pip install -U selenium
!pip install -U webdriver-manager
!pip install pandas

In [ ]:
from selenium import webdriver
import pandas as pd 
from lxml import html
import requests
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
import openpyxl
import os 
os.getcwd()
file = 'ProwessSeed.xlsx'
d1 = pd.ExcelFile(file)
print(d1.sheet_names)
df = d1.parse('Sheet1')
df.info
df.head(10)

In [ ]:
ps = openpyxl.load_workbook('ProwessSeed.xlsx')
sheet = ps['Sheet1']

In [ ]:
dx=[]
n=0
while(n<300):
    
    driver = webdriver.Chrome('C:/Users/shrut/Downloads/CD/chromedriver.exe')
    driver.get("https://www.vccedge.com/login.php")
    driver.find_element(By.XPATH,"//*[@id='edit-name']").send_keys('Jaya_Lakshmi@isb.edu')

    driver.find_element(By.XPATH,"//*[@id='edit-pass']").send_keys('isblrc@21')
    driver.find_element(By.XPATH,"//*[@id='user-login']/div[3]/button").click()
    WebDriverWait(driver, 10)

    print("N",n)
    for row in range(n+1,n+101):
        print("row",row)
        # Get CIN Number from Excel
        CIN = sheet['A' + str(row)].value
        WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/header/nav/div/div[2]/div/div/div/div[2]/div/input"))).send_keys(CIN)
        driver.find_element(By.XPATH,"//*[@id='header_nav_toolbar']/div/div/div[3]/div/button").click()
        try:
            WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div/div/section/div/div/div[2]/div/div[1]/div/div[2]/div[2]/div[2]/div/div[1]/div/a"))).click()
            driver.switch_to.window(driver.window_handles[1])
            # year
            try:
                objYear=driver.find_element(By.XPATH,"//*[@id='home']/div/table/tbody/tr[1]/td[1]")
                year=objYear.text
                print(year)      
            except:
                year=""       
            try:
                objnetSales=driver.find_element(By.XPATH,"//*[@id='home']/div/table/tbody/tr[1]/td[2]")
                netSales=objnetSales.text.replace(',',"")
                print(netSales)
            except:
                netSales=""
            try:
                objTotalIncome=driver.find_element(By.XPATH,"//*[@id='home']/div/table/tbody/tr[1]/td[3]")
                TotalIncome=objTotalIncome.text.replace(',',"")
                print(TotalIncome)
            except:
                TotalIncome=""          
            try:
                objEBITDA=driver.find_element(By.XPATH,"//*[@id='home']/div/table/tbody/tr[1]/td[4]")
                EBITDA=objEBITDA.text.replace(',',"")
                print(EBITDA)
            except:
                EBITDA=""              
            try:
                objPAT=driver.find_element(By.XPATH,"//*[@id='home']/div/table/tbody/tr[1]/td[5]")
                PAT=objPAT.text.replace(',',"")
                print(PAT)   
            except:
                PAT=""       
            try:
                objEquityPaidUP=driver.find_element(By.XPATH,"//*[@id='home']/div/table/tbody/tr[1]/td[6]")
                EquityPaidUP=objEquityPaidUP.text.replace(',',"")
                print(EquityPaidUP)       
            except:
                EquityPaidUP=""
            try:
                objTotalDebt=driver.find_element(By.XPATH,"//*[@id='home']/div/table/tbody/tr[1]/td[7]")
                TotalDebt=objTotalDebt.text.replace(',',"")
                print(TotalDebt)       
            except:
                TotalDebt=""

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            dx.insert(row,[CIN,year,netSales,TotalIncome,EBITDA,PAT,EquityPaidUP,TotalDebt])
            continue
        except:
            print("Company Not found",CIN)
            dx.insert(row,[CIN,"","","","","","",""])
            driver.get("https://app.vccedge.com/home")
        
    driver.close()

    df=pd.DataFrame(dx,columns=['CIN','Year','NetSales','TotalIncome','EBITDA','PAT','EquityPaidUP','TotalDebt'])
    df
    df.to_excel('outputSeleniumProwess.xlsx', index=False, na_rep = 'NA', header = True)
    n=n+100
